In [1]:
import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import unicodedata
import re
import time

with open('./spa-eng/spa.txt') as f:
    spa_eng_list = f.read().strip().split('\n')

len(spa_eng_list)

123376

In [2]:
# sample size
num_examples = 10000

# creates lists containing each pair
original_word_pairs = [[w for w in l.split('\t')] for l in spa_eng_list[:num_examples]]

data = pd.DataFrame(original_word_pairs, columns=["eng", "es"])

data.sample(10)

,eng,es
2216,Now move on.,Ahora prosigue.
6843,Never say that.,Nunca digas eso.
5952,Everybody paid.,Todos pagaron.
5625,Where are you?,¿Dónde estás?
3622,Tom likes me.,Le gusto a Tom.
1698,Be merciful.,Sé clemente.
7562,Who wants that?,¿Quién quiere eso?
3183,Is this mine?,¿Es éste el mío?
9517,Tom called Mary.,Tom llamó a Mary.
8532,I want my money.,Quiero mi plata.


In [3]:
# utils.py

# Converts the unicode file to ascii
def unicode_to_ascii(s):
    """
    Normalizes latin chars with accent to their canonical decomposition
    """
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ." 
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    
    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

def max_length(tensor):
    return max(len(t) for t in tensor)

def pad_sequences(x, max_len):
    padded = np.zeros((max_len), dtype=np.int64)
    if len(x) > max_len:
        padded[:] = x[:max_len]
    else:
        padded[:len(x)] = x
    return padded

# sort batch function to be able to use with pad_packed_sequence
def sort_batch(X, y, lengths):
    lengths, indx = lengths.sort(dim=0, descending=True)
    X = X[indx]
    y = y[indx]
    return X.transpose(0,1), y, lengths # transpose (batch x seq) to (seq x batch)

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count  

In [4]:
# process the data
data["eng"] = data.eng.apply(lambda w: preprocess_sentence(w))
data["es"] = data.es.apply(lambda w: preprocess_sentence(w))
data.sample(10)

,eng,es
9046,<start> my shoes squeak . <end>,<start> mis zapatos chirrian . <end>
8894,<start> it s not my car . <end>,<start> no es mi auto . <end>
7445,<start> we need a plan . <end>,<start> necesitamos un plan . <end>
1108,<start> hold still . <end>,<start> mantente quieto . <end>
1612,<start> we want it . <end>,<start> lo queremos . <end>
8628,<start> i m a christian . <end>,<start> soy cristiano . <end>
4482,<start> i let you win . <end>,<start> yo te dejo ganar . <end>
4396,<start> i fell for it . <end>,<start> me enganaron a causa de eso . <end>
5324,<start> this is japan . <end>,<start> esto es el japon . <end>
7961,<start> even tom smiled . <end>,<start> incluso tom sonrio . <end>


In [5]:
# build the vocabulary
class LanguageIndex():
    
    def __init__(self, lang):
        """ lang: the list of phrases from each language """
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()
        
        self.create_index()
        
    def create_index(self):
        for phrase in self.lang:
            # update with individual tokens
            self.vocab.update(phrase.split(' '))
            
        # sort the vocab
        self.vocab = sorted(self.vocab)

        # add a padding token with index 0
        self.word2idx['<pad>'] = 0
        
        # word to index mapping
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1 # +1 because of pad token
        
        # index to word mapping
        for word, index in self.word2idx.items():
            self.idx2word[index] = word

In [6]:
# index language using the class above
inp_lang = LanguageIndex(data["es"].values.tolist())
targ_lang = LanguageIndex(data["eng"].values.tolist())

# Vectorize the input and target languages
input_tensor = [[inp_lang.word2idx[s] for s in es.split(' ')]  for es in data["es"].values.tolist()]
target_tensor = [[targ_lang.word2idx[s] for s in eng.split(' ')]  for eng in data["eng"].values.tolist()]
print(input_tensor[:5])
print(target_tensor[:5])

# calculate the max_length of input and output tensor
max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
print('max_length_inp: %d'%max_length_inp)
print('max_length_tar: %d'%max_length_tar)

# inplace padding
input_tensor = [pad_sequences(x, max_length_inp) for x in input_tensor]
target_tensor = [pad_sequences(x, max_length_tar) for x in target_tensor]
print('len(input_tensor): %d'%len(input_tensor))
print('len(target_tensor): %d'%len(target_tensor))

# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)
print('len(input_tensor_train): %d\nlen(target_tensor_train): %d\nlen(input_tensor_val): %d\nlen(target_tensor_val): %d\n'
       % (len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)))

[[5, 4361, 3, 4], [5, 4425, 3, 4], [5, 4353, 3, 4], [5, 4360, 3, 4], [5, 2221, 3, 4]]
[[5, 843, 3, 4], [5, 843, 3, 4], [5, 843, 3, 4], [5, 843, 3, 4], [5, 944, 3, 4]]
max_length_inp: 12
max_length_tar: 8
len(input_tensor): 10000
len(target_tensor): 10000
len(input_tensor_train): 8000
len(target_tensor_train): 8000
len(input_tensor_val): 2000
len(target_tensor_val): 2000



In [7]:
len(targ_lang.vocab)

2285

In [8]:
# conver the data to tensors and pass to the Dataloader to create an batch iterator
class IterData(Dataset):
    
    def __init__(self, X, y):
        self.data = X
        self.target = y
        self.length = [ np.sum(1 - np.equal(x, 0)) for x in X]
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        x_len = self.length[index]
        return x, y, x_len
    
    def __len__(self):
        return len(self.data)

In [9]:
# hyperparams
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 16
N_BATCH = BUFFER_SIZE // BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

train_dataset = IterData(input_tensor_train, target_tensor_train)
val_dataset = IterData(input_tensor_val, target_tensor_val)

dataset_train = DataLoader(train_dataset, batch_size = BATCH_SIZE, 
                     drop_last=True,
                     shuffle=True)

dataset_val = DataLoader(val_dataset, batch_size = 1, 
                     drop_last=True,
                     shuffle=False)

In [10]:
# model.py

class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = nn.GRU(self.embedding_dim, self.enc_units)
        
    def forward(self, x, lens,  device):
        # x: (batch_size, max_length, embedding_dim)
        x = self.embedding(x) 
        x = pack_padded_sequence(x, lens)
        
        self.hidden = self.initialize_hidden_state(device)
        
        output, self.hidden = self.gru(x, self.hidden)
        output, _ = pad_packed_sequence(output)
        
        return output, self.hidden
        
    def initialize_hidden_state(self, device):
        return torch.zeros((1, self.batch_size, self.enc_units)).to(device)
    
    
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dec_units, enc_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.enc_units = enc_units
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = nn.GRU(self.embedding_dim + self.enc_units,
                                         self.dec_units,
                                         batch_first=True)
        self.fc = nn.Linear(self.enc_units, self.vocab_size)
        
        # for attention
        self.W1 = nn.Linear(self.enc_units, self.dec_units)
        self.W2 = nn.Linear(self.enc_units, self.dec_units)
        self.V = nn.Linear(self.enc_units, 1)
        
    def forward(self, x, hidden, enc_output):
        # enc_output original: (max_length, batch_size, enc_units)
        # enc_output converted == (batch_size, max_length, hidden_size)
        enc_output = enc_output.permute(1, 0, 2)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        hidden_with_time_axis = hidden.permute(1, 0, 2)
        
        # score: (batch_size, max_length, hidden_size)
        score = torch.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis))
        
        # attention_weights shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        attention_weights = torch.softmax(self.V(score), dim=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = torch.sum(context_vector, dim=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        # Looks like attention vector in diagram of source
        x = torch.cat((context_vector.unsqueeze(1), x), -1)
        
        # passing the concatenated vector to the GRU
        # output: (batch_size, 1, hidden_size)
        output, state = self.gru(x)
            
        # output shape == (batch_size * 1, hidden_size)
        output =  output.view(-1, output.size(2))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
    
    def initialize_hidden_state(self):
        return torch.zeros((1, self.batch_sz, self.dec_units))

In [11]:
criterion = nn.CrossEntropyLoss()

def loss_fuction(real, pred):
    """ Only consider non-zero inputs in the loss; mask needed """
    #mask = 1 - np.equal(real, 0) # assign 0 to all above 0 and 1 to all 0s
    #print(mask)
    mask = real.ge(1).type(torch.FloatTensor)
#     mask = real.ge(1).type(torch.cuda.FloatTensor)
    
    loss_ = criterion(pred, real) * mask 
    return torch.mean(loss_)

In [12]:
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(' Train on ' + str(device).upper())

PRE_TRAINED = True

encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, units, BATCH_SIZE)

if PRE_TRAINED:
    encoder.load_state_dict(torch.load('eng2spa_encoder_params.pkl'))
    decoder.load_state_dict(torch.load('eng2spa_decoder_params.pkl'))

encoder.to(device)
decoder.to(device)

optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=0.001)

 Train on CPU


In [13]:
# train.py

def train(encoder, decoder, epoch):
    losses = AverageMeter()
    batch_time = AverageMeter()

    encoder.train()
    decoder.train()

    total_loss = 0

    for batch, (inp, targ, inp_len) in enumerate(dataset_train):
        loss = 0

        xs, ys, lens = sort_batch(inp, targ, inp_len)
        enc_output, enc_hidden = encoder(xs.to(device), lens, device)
        dec_hidden = enc_hidden
        dec_input = torch.tensor([[targ_lang.word2idx['<start>']]] * BATCH_SIZE)
        
        end = time.time()

        for t in range(1, ys.size(1)):
            predictions, dec_hidden, _ = decoder(dec_input.to(device),
                                                                            dec_hidden.to(device),
                                                                            enc_output.to(device))
            loss += loss_fuction(ys[:, t].to(device), predictions.to(device))
            dec_input = ys[:, t].unsqueeze(1)

        batch_loss = (loss / int(ys.size(1)))
        losses.update(batch_loss, xs.size(0))
        
        total_loss += batch_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        batch_time.update(time.time() - end)
        end = time.time()

        if batch % PRINT_FREQ == 0:
                print('Epoch: [{0}] [{1}/{2}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      .format(
                       epoch, batch, len(dataset_train), batch_time=batch_time, loss=losses))
                
#         print('Saving Model...')
#         torch.save(encoder.state_dict(), 'eng2spa_encoder_params.pkl')
#         torch.save(decoder.state_dict(), 'eng2spa_decoder_params.pkl')
#         print('Model Saved Successfully!')
        if batch % 200 == 0:
            print('Saving Model...')
            torch.save(encoder.state_dict(), 'eng2spa_encoder_params.pkl')
            torch.save(decoder.state_dict(), 'eng2spa_decoder_params.pkl')
            print('Model Saved Successfully!')

In [14]:
def validate(encoder, decoder):
    encoder.eval()
    decoder.eval()
    acc = 0
    for batch, (inp, targ, inp_len) in enumerate(dataset_val):
        xs, ys, lens = sort_batch(inp, targ, inp_len)
        raw_sen = xs.numpy()
        print(raw_sen.shape)
        for raw_word in raw_sen:
#             print(raw_word)
            print(inp_lang.idx2word[int(raw_word)], end=' ')
        print()
        raw_sen1 = ys.numpy()[0]
        print(raw_sen1.shape)
        for raw_word1 in raw_sen1:
#             print(raw_word)
            print(targ_lang.idx2word[int(raw_word1)], end=' ')
        print()
        input()
        enc_output, enc_hidden = encoder(xs.to(device), lens, device)
        dec_hidden = enc_hidden
        dec_input = torch.tensor([[targ_lang.word2idx['<start>']]] * 1)
        for t in range(1, ys.size(1)):
            print(ys.size(1))
            predictions, dec_hidden, _ = decoder(dec_input.to(device),
                                                                            dec_hidden.to(device),
                                                                            enc_output.to(device))
#             print(predictions.shape)
#             print(ys[:,t])
            pred = torch.argmax(predictions)
#             print(predictions.detach().numpy())
#             print(predictions.detach().numpy()[0][3])
#             print(int(pred.numpy()))
            print(targ_lang.idx2word[int(pred.numpy())], end=' ')
        print()
#             input()
#         print(predictions)

In [15]:
def apply(encoder, decoder):
    encoder.eval()
    decoder.eval()
    x = input('Input an Spanish sentence to translate: ')
    

In [16]:
# model train

EPOCH = 10
PRINT_FREQ = 5

for epoch in range(EPOCH):
    train(encoder, decoder, epoch)
    print('Saving Model...')
    torch.save(encoder.state_dict(), 'eng2spa_encoder_params.pkl')
    torch.save(decoder.state_dict(), 'eng2spa_decoder_params.pkl')
    print('Model Saved Successfully!')

Epoch: [0] [0/500]	Time 0.765 (0.765)	Loss 5.3143 (5.3143)	
Saving Model...
Model Saved Successfully!
Epoch: [0] [5/500]	Time 0.577 (0.610)	Loss 3.2598 (4.1514)	
Epoch: [0] [10/500]	Time 0.705 (0.632)	Loss 2.8092 (3.6033)	
Epoch: [0] [15/500]	Time 0.551 (0.689)	Loss 2.6122 (3.3181)	
Epoch: [0] [20/500]	Time 0.605 (0.665)	Loss 2.7339 (3.1550)	
Epoch: [0] [25/500]	Time 0.708 (0.660)	Loss 2.4596 (3.0170)	
Epoch: [0] [30/500]	Time 0.743 (0.653)	Loss 2.4999 (2.9261)	
Epoch: [0] [35/500]	Time 0.709 (0.654)	Loss 2.4487 (2.8500)	
Epoch: [0] [40/500]	Time 0.826 (0.670)	Loss 2.1671 (2.7750)	
Epoch: [0] [45/500]	Time 0.761 (0.668)	Loss 2.3109 (2.7176)	
Epoch: [0] [50/500]	Time 0.545 (0.657)	Loss 2.2816 (2.6736)	
Epoch: [0] [55/500]	Time 0.536 (0.650)	Loss 2.1968 (2.6231)	
Epoch: [0] [60/500]	Time 0.528 (0.641)	Loss 2.1722 (2.5816)	
Epoch: [0] [65/500]	Time 0.544 (0.634)	Loss 2.1068 (2.5453)	
Epoch: [0] [70/500]	Time 0.552 (0.628)	Loss 1.9087 (2.5130)	
Epoch: [0] [75/500]	Time 0.575 (0.624)	Loss 2

Epoch: [1] [150/500]	Time 0.554 (0.561)	Loss 0.8190 (0.9004)	
Epoch: [1] [155/500]	Time 0.533 (0.561)	Loss 0.8236 (0.9027)	
Epoch: [1] [160/500]	Time 0.540 (0.561)	Loss 0.9452 (0.9044)	
Epoch: [1] [165/500]	Time 0.539 (0.560)	Loss 0.7940 (0.9018)	
Epoch: [1] [170/500]	Time 0.529 (0.560)	Loss 0.8942 (0.9009)	
Epoch: [1] [175/500]	Time 0.742 (0.561)	Loss 0.9095 (0.9029)	
Epoch: [1] [180/500]	Time 0.551 (0.563)	Loss 0.8019 (0.9025)	
Epoch: [1] [185/500]	Time 0.550 (0.563)	Loss 0.8561 (0.9024)	
Epoch: [1] [190/500]	Time 0.551 (0.563)	Loss 0.8122 (0.9004)	
Epoch: [1] [195/500]	Time 0.549 (0.563)	Loss 0.9398 (0.9025)	
Epoch: [1] [200/500]	Time 0.525 (0.562)	Loss 1.0656 (0.9029)	
Saving Model...
Model Saved Successfully!
Epoch: [1] [205/500]	Time 0.635 (0.564)	Loss 0.6626 (0.9005)	
Epoch: [1] [210/500]	Time 0.540 (0.563)	Loss 0.9163 (0.8993)	
Epoch: [1] [215/500]	Time 0.559 (0.563)	Loss 0.8854 (0.9018)	
Epoch: [1] [220/500]	Time 0.551 (0.563)	Loss 0.6204 (0.9021)	
Epoch: [1] [225/500]	Time 0.

Epoch: [2] [300/500]	Time 0.567 (0.562)	Loss 0.5585 (0.4407)	
Epoch: [2] [305/500]	Time 0.591 (0.562)	Loss 0.4690 (0.4404)	
Epoch: [2] [310/500]	Time 0.561 (0.562)	Loss 0.5218 (0.4404)	
Epoch: [2] [315/500]	Time 0.545 (0.562)	Loss 0.4070 (0.4409)	
Epoch: [2] [320/500]	Time 0.532 (0.561)	Loss 0.4647 (0.4416)	
Epoch: [2] [325/500]	Time 0.532 (0.561)	Loss 0.4460 (0.4418)	
Epoch: [2] [330/500]	Time 0.549 (0.561)	Loss 0.3226 (0.4413)	
Epoch: [2] [335/500]	Time 0.528 (0.561)	Loss 0.3429 (0.4413)	
Epoch: [2] [340/500]	Time 0.543 (0.561)	Loss 0.5324 (0.4425)	
Epoch: [2] [345/500]	Time 0.544 (0.561)	Loss 0.5539 (0.4438)	
Epoch: [2] [350/500]	Time 0.597 (0.561)	Loss 0.5231 (0.4440)	
Epoch: [2] [355/500]	Time 0.577 (0.560)	Loss 0.3760 (0.4442)	
Epoch: [2] [360/500]	Time 0.566 (0.560)	Loss 0.3988 (0.4438)	
Epoch: [2] [365/500]	Time 0.553 (0.560)	Loss 0.3536 (0.4440)	
Epoch: [2] [370/500]	Time 0.531 (0.560)	Loss 0.3266 (0.4441)	
Epoch: [2] [375/500]	Time 0.548 (0.560)	Loss 0.4939 (0.4441)	
Epoch: [

Epoch: [3] [450/500]	Time 0.552 (0.556)	Loss 0.3304 (0.2292)	
Epoch: [3] [455/500]	Time 0.547 (0.556)	Loss 0.3796 (0.2297)	
Epoch: [3] [460/500]	Time 0.567 (0.556)	Loss 0.3220 (0.2309)	
Epoch: [3] [465/500]	Time 0.548 (0.556)	Loss 0.1800 (0.2311)	
Epoch: [3] [470/500]	Time 0.530 (0.556)	Loss 0.1930 (0.2316)	
Epoch: [3] [475/500]	Time 0.522 (0.556)	Loss 0.2083 (0.2317)	
Epoch: [3] [480/500]	Time 0.540 (0.556)	Loss 0.4096 (0.2320)	
Epoch: [3] [485/500]	Time 0.557 (0.556)	Loss 0.2083 (0.2319)	
Epoch: [3] [490/500]	Time 0.556 (0.556)	Loss 0.2753 (0.2322)	
Epoch: [3] [495/500]	Time 0.580 (0.556)	Loss 0.3669 (0.2329)	
Saving Model...
Model Saved Successfully!
Epoch: [4] [0/500]	Time 0.552 (0.552)	Loss 0.0983 (0.0983)	
Saving Model...
Model Saved Successfully!
Epoch: [4] [5/500]	Time 0.543 (0.544)	Loss 0.1337 (0.1213)	
Epoch: [4] [10/500]	Time 0.545 (0.546)	Loss 0.1755 (0.1527)	
Epoch: [4] [15/500]	Time 0.562 (0.552)	Loss 0.1065 (0.1481)	
Epoch: [4] [20/500]	Time 0.530 (0.549)	Loss 0.1268 (0.

Epoch: [5] [95/500]	Time 0.552 (0.567)	Loss 0.1842 (0.1036)	
Epoch: [5] [100/500]	Time 0.551 (0.567)	Loss 0.1029 (0.1041)	
Epoch: [5] [105/500]	Time 0.536 (0.567)	Loss 0.0647 (0.1033)	
Epoch: [5] [110/500]	Time 0.526 (0.566)	Loss 0.1316 (0.1026)	
Epoch: [5] [115/500]	Time 0.522 (0.565)	Loss 0.0892 (0.1026)	
Epoch: [5] [120/500]	Time 0.522 (0.564)	Loss 0.0514 (0.1024)	
Epoch: [5] [125/500]	Time 0.540 (0.564)	Loss 0.0386 (0.1037)	
Epoch: [5] [130/500]	Time 0.522 (0.565)	Loss 0.1044 (0.1037)	
Epoch: [5] [135/500]	Time 0.538 (0.564)	Loss 0.0859 (0.1035)	
Epoch: [5] [140/500]	Time 0.572 (0.563)	Loss 0.2812 (0.1054)	
Epoch: [5] [145/500]	Time 0.550 (0.565)	Loss 0.1573 (0.1061)	
Epoch: [5] [150/500]	Time 0.551 (0.565)	Loss 0.1547 (0.1070)	
Epoch: [5] [155/500]	Time 0.622 (0.566)	Loss 0.0888 (0.1066)	
Epoch: [5] [160/500]	Time 0.544 (0.567)	Loss 0.1367 (0.1068)	
Epoch: [5] [165/500]	Time 0.587 (0.568)	Loss 0.1538 (0.1072)	
Epoch: [5] [170/500]	Time 0.569 (0.573)	Loss 0.1333 (0.1064)	
Epoch: [5

Epoch: [6] [245/500]	Time 0.544 (0.558)	Loss 0.0446 (0.0932)	
Epoch: [6] [250/500]	Time 0.525 (0.559)	Loss 0.0731 (0.0933)	
Epoch: [6] [255/500]	Time 0.527 (0.559)	Loss 0.1209 (0.0934)	
Epoch: [6] [260/500]	Time 0.543 (0.559)	Loss 0.1460 (0.0932)	
Epoch: [6] [265/500]	Time 0.543 (0.559)	Loss 0.0769 (0.0931)	
Epoch: [6] [270/500]	Time 0.529 (0.559)	Loss 0.0545 (0.0936)	
Epoch: [6] [275/500]	Time 0.572 (0.559)	Loss 0.0367 (0.0935)	
Epoch: [6] [280/500]	Time 0.527 (0.559)	Loss 0.0660 (0.0936)	
Epoch: [6] [285/500]	Time 0.580 (0.560)	Loss 0.1028 (0.0941)	
Epoch: [6] [290/500]	Time 0.554 (0.560)	Loss 0.0275 (0.0937)	
Epoch: [6] [295/500]	Time 0.556 (0.560)	Loss 0.0711 (0.0937)	
Epoch: [6] [300/500]	Time 0.558 (0.560)	Loss 0.1225 (0.0942)	
Epoch: [6] [305/500]	Time 0.587 (0.560)	Loss 0.2164 (0.0942)	
Epoch: [6] [310/500]	Time 0.566 (0.560)	Loss 0.0596 (0.0938)	
Epoch: [6] [315/500]	Time 0.556 (0.561)	Loss 0.0861 (0.0938)	
Epoch: [6] [320/500]	Time 0.569 (0.561)	Loss 0.1567 (0.0945)	
Epoch: [

Epoch: [7] [395/500]	Time 0.766 (0.671)	Loss 0.0785 (0.0862)	
Epoch: [7] [400/500]	Time 0.735 (0.673)	Loss 0.2632 (0.0864)	
Saving Model...
Model Saved Successfully!
Epoch: [7] [405/500]	Time 0.747 (0.674)	Loss 0.2733 (0.0868)	
Epoch: [7] [410/500]	Time 0.802 (0.675)	Loss 0.0954 (0.0868)	
Epoch: [7] [415/500]	Time 0.752 (0.676)	Loss 0.1155 (0.0865)	
Epoch: [7] [420/500]	Time 0.760 (0.677)	Loss 0.1914 (0.0874)	
Epoch: [7] [425/500]	Time 0.561 (0.677)	Loss 0.0267 (0.0877)	
Epoch: [7] [430/500]	Time 1.040 (0.677)	Loss 0.0446 (0.0878)	
Epoch: [7] [435/500]	Time 0.524 (0.677)	Loss 0.2326 (0.0887)	
Epoch: [7] [440/500]	Time 0.553 (0.675)	Loss 0.0755 (0.0888)	
Epoch: [7] [445/500]	Time 0.542 (0.674)	Loss 0.2511 (0.0895)	
Epoch: [7] [450/500]	Time 0.804 (0.674)	Loss 0.1772 (0.0898)	
Epoch: [7] [455/500]	Time 0.579 (0.674)	Loss 0.0989 (0.0900)	
Epoch: [7] [460/500]	Time 0.759 (0.675)	Loss 0.1391 (0.0903)	
Epoch: [7] [465/500]	Time 0.804 (0.675)	Loss 0.0841 (0.0901)	
Epoch: [7] [470/500]	Time 0.

Epoch: [9] [35/500]	Time 0.559 (0.554)	Loss 0.0579 (0.0833)	
Epoch: [9] [40/500]	Time 0.531 (0.553)	Loss 0.0450 (0.0795)	
Epoch: [9] [45/500]	Time 0.572 (0.553)	Loss 0.0446 (0.0812)	
Epoch: [9] [50/500]	Time 0.558 (0.553)	Loss 0.0873 (0.0834)	
Epoch: [9] [55/500]	Time 0.571 (0.553)	Loss 0.0685 (0.0810)	
Epoch: [9] [60/500]	Time 0.552 (0.553)	Loss 0.0525 (0.0776)	
Epoch: [9] [65/500]	Time 0.564 (0.553)	Loss 0.0876 (0.0777)	
Epoch: [9] [70/500]	Time 0.547 (0.553)	Loss 0.0484 (0.0758)	
Epoch: [9] [75/500]	Time 0.543 (0.553)	Loss 0.1028 (0.0785)	
Epoch: [9] [80/500]	Time 0.525 (0.552)	Loss 0.0708 (0.0766)	
Epoch: [9] [85/500]	Time 0.536 (0.552)	Loss 0.1093 (0.0787)	
Epoch: [9] [90/500]	Time 0.537 (0.552)	Loss 0.0466 (0.0791)	
Epoch: [9] [95/500]	Time 0.569 (0.552)	Loss 0.1746 (0.0806)	
Epoch: [9] [100/500]	Time 0.554 (0.552)	Loss 0.0515 (0.0787)	
Epoch: [9] [105/500]	Time 0.618 (0.552)	Loss 0.0457 (0.0788)	
Epoch: [9] [110/500]	Time 0.546 (0.552)	Loss 0.0088 (0.0765)	
Epoch: [9] [115/500]	

In [18]:
encoder_val = Encoder(vocab_inp_size, embedding_dim, units, 1)
decoder_val = Decoder(vocab_tar_size, embedding_dim, units, units, 1)

encoder_val.to(device)
decoder_val.to(device)

# print(encoder_val)

encoder_val.load_state_dict(torch.load('eng2spa_encoder_params.pkl'))
decoder_val.load_state_dict(torch.load('eng2spa_decoder_params.pkl'))

# validate(encoder_val, decoder_val)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])